In [1]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["train"])
df

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...
...,...,...,...,...,...,...
11674,The enzyme pepsin plays an important role in t...,lipids,protons,proteins,peptides,Protein A large part of protein digestion take...
11675,What remains a constant of radioactive substan...,acidity,temperature,volatility,rate of decay,The rate of decay of a radioactive substance i...
11676,"Terrestrial ecosystems, also known for their d...",substrates,bisomes,monomes,biomes,"Terrestrial ecosystems, also known for their d..."
11677,High explosives create shock waves that exceed...,turbulence,light speed,ion speed,supersonic,The modern day formulation of gun powder is ca...


In [2]:
df.loc[0]

question          What type of organism is commonly used in prep...
distractor3                                                 viruses
distractor1                                                protozoa
distractor2                                             gymnosperms
correct_answer                                 mesophilic organisms
support           Mesophiles grow best in moderate temperature, ...
Name: 0, dtype: object

In [3]:
import torch
import os

from sentence_transformers import CrossEncoder
from src.chain_initialisation import init_chain
from src.embedding_management import init_embeddings, add_embeddings_from_files
from src.model_initialisation import init_model
from src.pipeline_initialisation import init_pipeline
from src.retriever_initialisation import init_retriever
from src.dynamic_doc_retrieval import download_documents, initialise_keyword_model, generate_query_from_question

2025-05-28 15:39:21.099936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748439561.497858   47729 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748439561.535620   47729 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748439561.631152   47729 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748439561.631244   47729 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748439561.631254   47729 computation_placer.cc:177] computation placer alr

In [4]:
BASE_DIR = os.curdir
DATA_DIR = os.path.join(BASE_DIR, "data")
DYNAMIC_DATA_DIR = os.path.join(DATA_DIR, "dynamic")
print(f"dynamic data dir: {DYNAMIC_DATA_DIR}")


def rerank_documents(reranker, query: str, documents: list, top_k: int = 3, min_score = 0.5) -> list:
    if not documents:
        return []

    pairs = [(query, doc.page_content) for doc in documents]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

    return [doc for doc, score in ranked[:top_k] if score > min_score]

def ask_question(model_chain, reranker, keyword_model, vectorstore, question, num_docs, max_tries, online, rerank_top_k, rerank_min_score):
    if online:
        print("Obtaining data from scholar...")
        query = generate_query_from_question(keyword_model, question)
        downloaded_files = download_documents(query, DYNAMIC_DATA_DIR, num_docs, max_tries)
        add_embeddings_from_files(vectorstore, downloaded_files)

    response = model_chain.invoke({
        "question": question,
        "chat_history": []
    })
    answer = response["answer"].split("### Answer:")[-1].strip()
    source_documents = response["source_documents"]
    filtered_docs = rerank_documents(reranker, question, source_documents, rerank_top_k, rerank_min_score)
    return answer, source_documents, filtered_docs

dynamic data dir: ./data/dynamic


In [5]:
cuda_available = torch.cuda.is_available()
print(f"Initializing model... CUDA available: {cuda_available}")
model, model_name = init_model(cuda_available)
pipeline = init_pipeline(model=model, model_name=model_name)
vectorstore = init_embeddings(cuda_available)
retriever = init_retriever(vectorstore)
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
chain = init_chain(pipeline=pipeline, retriever=retriever)
keyword_model = initialise_keyword_model()
print("Start-up complete.")

Initializing model... CUDA available: False


Device set to use cpu
/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/src/embedding_management.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


Loading FAISS from file - skipping embedding build.
Vectorstore loaded successfully.
Retriever initialized with k=2 and similarity threshold=0.8


/home/henri/ul-fri-nlp-course-project-2024-2025-drop-table-teams/src/chain_initialisation.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


LLM model chain created.
Start-up complete.


In [7]:
from sentence_transformers import SentenceTransformer, util
from fuzzywuzzy import fuzz

# Initialize embedding model
evaluation_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def evaluate_answer(row, chain, reranker, keyword_model, vectorstore, num_docs, max_tries, online, rerank_top_k, rerank_min_score):
    question = row['question']
    correct_answer = row['correct_answer']
    support = row['support']
    distractors = [row['distractor1'], row['distractor2'], row['distractor3']]

    # Chatbot answer
    response, source_docs, filtered_docs = ask_question(chain, reranker, keyword_model, vectorstore, question, num_docs, max_tries, online, rerank_top_k, rerank_min_score)
    response = response.lower()

    # Exact match
    exact = int(correct_answer.lower() in response)

    # Fuzzy match
    fuzzy_score = fuzz.partial_ratio(correct_answer.lower(), response)

    # Embedding similarity to correct answer
    emb_correct = evaluation_model.encode(correct_answer, convert_to_tensor=True)
    emb_response = evaluation_model.encode(response, convert_to_tensor=True)
    sim_correct = util.cos_sim(emb_correct, emb_response).item()

    # Embedding similarity to support passage
    emb_support = evaluation_model.encode(support, convert_to_tensor=True)
    sim_support = util.cos_sim(emb_support, emb_response).item()

    # Compare similarity to distractors
    sim_distractors = []
    for d in distractors:
        emb_distractor = evaluation_model.encode(d, convert_to_tensor=True)
        sim_distractors.append(util.cos_sim(emb_distractor, emb_response).item())
    
    # Is chatbot closer to correct than to all distractors?
    correct_beats_distractors = int(sim_correct > max(sim_distractors))

    return {
        "question": question,
        "chatbot_response": response,
        "exact_match": exact,
        "fuzzy_score": fuzzy_score,
        "sim_to_correct": round(sim_correct, 3),
        "sim_to_support": round(sim_support, 3),
        "correct_beats_distractors": correct_beats_distractors,
        "source_docs": source_docs,
        "filtered_docs": filtered_docs,
    }


In [14]:
for i, row in df[:3].iterrows():
    print(row["question"])

What type of organism is commonly used in preparation of foods such as cheese and yogurt?
What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?


In [ ]:

configs = [
    {"k": 3, "th": 0.7},
    {"k": 3, "th": 0.9},
    {"k": 5, "th": 0.7},
    {"k": 5, "th": 0.85},
    {"k": 10, "th": 0.7},
    {"k": 10, "th": 0.9},
]

rerank_configs = [
    {"top_k": 2, "min_score": 0.3},
    {"top_k": 3, "min_score": 0.5},
    {"top_k": 5, "min_score": 0.6},
]


for config in configs:
    retriever = init_retriever(vectorstore, k=config["k"], th=config["th"])
    reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    chain = init_chain(pipeline=pipeline, retriever=retriever)
    keyword_model = initialise_keyword_model()
    num_docs: int = int(os.getenv("num_docs"))
    max_tries: int = int(os.getenv("max_tries"))
    print("Start-up complete.")
    for online in [False, True]:
        for rerank_conf in rerank_configs:
            results = []
            filename = f"k_{config['k']}-thresh_{config['th']}-online_{online}-top_k_{rerank_conf['top_k']}-min_score_{rerank_conf['min_score']}.csv"
            for i, row in df[:20].iterrows():
                result = evaluate_answer(row, chain, reranker, keyword_model, vectorstore, num_docs, max_tries, online, rerank_conf["top_k"], rerank_conf["min_score"])
                results.append(result)
            
            results_df = pd.DataFrame(results)
            results_df.to_csv(f"{drive_path}/{filename}", index=False)

Retriever initialized with k=3 and similarity threshold=0.7
LLM model chain created.


/home/henri/.local/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='8f444c83-58e6-4f42-a652-38cac1d27915', metadata={'source': './data/dynamic/Yogurt_as_probiotic_carrier_food.pdf'}, page_content='sugar, fruit, ﬂavours, colouring, emulsiﬁers, stabilisers,\nand\nspeciﬁc\npure\ncultures\nof\nlactic\nacid\nbacteria\nA. Lourens-Hattingh, B.C. Viljoen / International Dairy Journal 11 (2001) 1–17\n7\n(S. thermophilus and L. bulgaricus) to conduct the\nfermentation process. The basic process of yogurt\nproduction is outlined in Fig. 3.\nS. thermophilus and L. bulgaricus exhibit a symbiotic\nrelationship during the processing of yogurt, with the\nratio between the species changing constantly (Radke-'), np.float32(-18.569504)), (Document(id='4a79e568-c772-47aa-aa72-df82ef8f3dc0', metadata={'source': './data/dynamic/Yogurt_as_probiotic_carrier_food.pdf'}, page_content='sugar, fruit, ﬂavours, colouring

Start-up complete.


/home/henri/.local/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='8f444c83-58e6-4f42-a652-38cac1d27915', metadata={'source': './data/dynamic/Yogurt_as_probiotic_carrier_food.pdf'}, page_content='sugar, fruit, ﬂavours, colouring, emulsiﬁers, stabilisers,\nand\nspeciﬁc\npure\ncultures\nof\nlactic\nacid\nbacteria\nA. Lourens-Hattingh, B.C. Viljoen / International Dairy Journal 11 (2001) 1–17\n7\n(S. thermophilus and L. bulgaricus) to conduct the\nfermentation process. The basic process of yogurt\nproduction is outlined in Fig. 3.\nS. thermophilus and L. bulgaricus exhibit a symbiotic\nrelationship during the processing of yogurt, with the\nratio between the species changing constantly (Radke-'), np.float32(-18.569504)), (Document(id='4a79e568-c772-47aa-aa72-df82ef8f3dc0', metadata={'source': './data/dynamic/Yogurt_as_probiotic_carrier_food.pdf'}, page_content='sugar, fruit, ﬂavours, colouring

IndexError: index out of range in self

In [17]:
if not os.path.exists("evaluation"):
    os.makedirs("evaluation")
results_df.to_csv(f"evaluation/{title}.csv", index=False)
results_df

,question,chatbot_response,exact_match,fuzzy_score,sim_to_correct,sim_to_support,correct_beats_distractors,source_docs,filtered_docs
0,What type of organism is commonly used in prep...,-------------------- -------- ---------- -----...,0,45,0.294,0.214,1,[],[]
1,What phenomenon makes global winds blow northe...,(t)he greatest hurricane of all times would ha...,0,60,0.101,0.194,0,[],[]
2,Changes from a less-ordered state to a more-or...,you can only switch between different states w...,0,50,0.006,0.207,0,[],[]
3,What is the least dangerous radioactive decay?,- there can be less than 0 contamination betwe...,0,36,0.160,0.445,1,[],[]
